# 1. Carregamento e caracterização dos dados

## Importa as bibliotecas necessárias

In [1]:
%pip install unidecode

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install plotly

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install nbformat --upgrade

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
import re
import nltk
import nbformat
import pandas as pd
import numpy as np
import tensorflow as tf
import plotly.express as px
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from unidecode import unidecode
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

scores = []

2024-07-04 23:23:31.951099: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-04 23:23:33.212652: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-04 23:23:34.095789: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 23:23:34.940035: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 23:23:34.943833: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-04 23:23:36.118638: I tensorflow/core/platform/cpu_feature_guard.cc:

In [8]:
# Baixa os recursos necessários do NLTK
nltk.download('stopwords')
nltk.download('rslp')

[nltk_data] Downloading package stopwords to /home/aluno/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /home/aluno/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

## Carrega o arquivo

In [9]:
df = pd.read_csv('AI_Human.csv', header=0, nrows=25_000)
df

,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0
...,...,...
24995,The power of music is something that has capti...,1.0
24996,The impact of technology on modern education i...,1.0
24997,The power of the written word is often underes...,1.0
24998,Sports nutrition is an important topic for ath...,1.0


## Checa se os dados estão balanceados

**0.0: Escrito por Humano**

**1.0: Escrito por AI**


In [10]:
fig = px.histogram(df, x='generated', template='simple_white', width=500, height=400)
fig.update_layout(bargap=0.2)
fig.show()

## Adiciona a quantidade de palavras que cada texto possui

In [11]:
df['quant_palavras'] = df['text'].apply(lambda x: len(x.split(" ")))
fig = px.histogram(df[df['quant_palavras'] < 1200], x='quant_palavras', template='simple_white', width=1200, height=800)
fig.show()

## Quantidade de textos fora do padrão

In [12]:
avg = np.average(df['quant_palavras'])
print("Média de palavras:", avg)
print("Quantidade de textos acima com no máximo o dobro de palavras da média:", df[(df['quant_palavras'] <= 2*avg) == True].count()[0])
print("Quantidade de textos acima com mais palavras que o dobro da média:", df[(df['quant_palavras'] > 2*avg) == True].count()[0])

Média de palavras: 375.99928
Quantidade de textos acima com no máximo o dobro de palavras da média: 24149
Quantidade de textos acima com mais palavras que o dobro da média: 851


/tmp/ipykernel_27409/2245895312.py:3: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/tmp/ipykernel_27409/2245895312.py:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



# 2. Limpeza e pré processamento dos dados

In [13]:
# Função para limpeza e stemização de texto
def limpar_texto_stem(texto):
    # Remove acentos
    texto = unidecode(texto)

    # Remove números, caracteres especiais e converte para minúsculas
    texto_limpo = re.sub(r'[^a-zA-Z\s]', '', texto).lower()

    # Tokenização
    palavras = texto_limpo.split()

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    palavras_sem_stopwords = [palavra for palavra in palavras if palavra not in stop_words]

    # Stemização
    stemmer = RSLPStemmer()
    palavras_stemizadas = [stemmer.stem(palavra) for palavra in palavras_sem_stopwords]

    return ' '.join(palavras_stemizadas)

In [14]:
df['text'] = df.apply(lambda row: limpar_texto_stem(row['text']), axis=1)
df

,text,generated,quant_palavras
0,car car around sinc becam fam henry ford creat...,0.0,581
1,transportation larg necessity countri worldwid...,0.0,458
2,amer lov affa vehicl se cooling say elisabeth ...,0.0,740
3,often rid car driv one motor vehicl work stor ...,0.0,684
4,car wonderful thing perhap one world greatest ...,0.0,868
...,...,...,...
24995,pow music something captivated us centuri day ...,1.0,342
24996,impact technology modern education undeniabl r...,1.0,353
24997,pow written word often underestimated many wri...,1.0,351
24998,sport nutrition important topic athlet level w...,1.0,327


# 3. Treinamento e avaliação dos modelos

## Divide os dados de treino e teste

In [15]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['generated'], test_size=0.3, random_state=42)

## BoW + kNN

In [16]:
# Cria a pipeline
pipe = Pipeline([
    ('vect', CountVectorizer()),
    ('knn', KNeighborsClassifier())
])

# Treina a pipeline
pipe.fit(X_train, y_train)

# Define os parâmetros a serem testados
param_grid = {
    'knn__n_neighbors': np.arange(1, 10),
    'knn__metric': ['euclidean', 'manhattan', 'minkowski']
}

# Configurar o GridSearchCV
grid_search = GridSearchCV(pipe, param_grid, cv=5, verbose=1, scoring='accuracy')

# Executar o GridSearchCV
grid_search.fit(X_train, y_train)

# Melhores parâmetros encontrados
print("Melhores parâmetros:", grid_search.best_params_)
print("Melhor acurácia:", grid_search.best_score_)

# Avaliar o modelo com os melhores parâmetros no conjunto de teste
test_accuracy = grid_search.score(X_test, y_test)
print("Acurácia no conjunto de teste:", test_accuracy)

scores.append({'modelo': 'BoW + kNN', 'accuracy': test_accuracy})

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Melhores parâmetros: {'knn__metric': 'euclidean', 'knn__n_neighbors': 2}
Melhor acurácia: 0.9374857142857144
Acurácia no conjunto de teste: 0.9385333333333333


## TF-IDF + kNN

In [17]:
# Cria a pipeline
pipe = Pipeline([
    ('vect', TfidfVectorizer()),
    ('knn', KNeighborsClassifier())
])

# Treina a pipeline
pipe.fit(X_train, y_train)

# Define os parâmetros a serem testados
param_grid = {
    'knn__n_neighbors': np.arange(1, 10),
    'knn__metric': ['euclidean', 'manhattan', 'minkowski']
}

# Configurar o GridSearchCV
grid_search = GridSearchCV(pipe, param_grid, cv=5, verbose=1, scoring='accuracy')

# Executar o GridSearchCV
grid_search.fit(X_train, y_train)

# Melhores parâmetros encontrados
print("Melhores parâmetros:", grid_search.best_params_)
print("Melhor acurácia:", grid_search.best_score_)

# Avaliar o modelo com os melhores parâmetros no conjunto de teste
test_accuracy = grid_search.score(X_test, y_test)
print("Acurácia no conjunto de teste:", test_accuracy)

scores.append({'modelo': 'TF-IDF + kNN', 'accuracy': test_accuracy})

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Melhores parâmetros: {'knn__metric': 'euclidean', 'knn__n_neighbors': 9}
Melhor acurácia: 0.9572571428571429
Acurácia no conjunto de teste: 0.9574666666666667


## BoW + MLP

In [18]:
# Carrega o dataset
X = df['text']
y = df['generated']

# Vetoriza usando Bag of Words
vectorizer = CountVectorizer()
X_vect = vectorizer.fit_transform(X)

# Transforma as labels em categorias
y = tf.keras.utils.to_categorical(y, num_classes=2)

# Divide os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_vect, y, test_size=0.2, random_state=42)

# Normaliza os dados
scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Cria o modelo de rede neural
model = Sequential([
    Dense(10, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(10, activation='relu'),
    Dense(10, activation='relu'),
    Dense(10, activation='relu'),
    Dense(10, activation='relu'),
    Dense(2, activation='softmax')
])

# Compilar o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(X_train_scaled, y_train, epochs=150, verbose=1)

# Avaliar o modelo no conjunto de teste
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f'Teste de Acurácia: {accuracy:.2f}')

scores.append({'modelo': 'BoW + MLP', 'accuracy': accuracy})

/home/aluno/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.9293 - loss: 0.1615
Epoch 2/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9990 - loss: 0.0048
Epoch 3/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9999 - loss: 5.1877e-04
Epoch 4/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 1.0000 - loss: 1.7984e-04
Epoch 5/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 1.0000 - loss: 9.9847e-05
Epoch 6/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 1.0000 - loss: 8.0532e-05
Epoch 7/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 1.0000 - loss: 2.8143e-05
Epoch 8/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 1.0000 - loss: 2.2641e-05
Epoch 9/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 1.0000 - loss: 9.6466e-06
Epoch 10/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 1.0000 - loss: 7.4222e-06
Epoch 11/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 1.0000 - loss: 3.5573e-06
Epoch

## TF-IDF + MLP

In [22]:
# Carrega o dataset
X = df['text']
y = df['generated']

# Vetoriza usando TF-IDF
vectorizer = TfidfVectorizer()
X_vect = vectorizer.fit_transform(X)

# Transforma as labels em categorias
y = tf.keras.utils.to_categorical(y, num_classes=2)

# Divide os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_vect, y, test_size=0.2, random_state=42)

# Normaliza os dados
scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Cria o modelo de rede neural
model = Sequential([
    Dense(10, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(10, activation='relu'),
    Dense(10, activation='relu'),
    Dense(10, activation='relu'),
    Dense(10, activation='relu'),
    Dense(2, activation='softmax')
])

# Compilar o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(X_train_scaled, y_train, epochs=150, verbose=1)

# Avaliar o modelo no conjunto de teste
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f'Teste de Acurácia: {accuracy:.2f}')

scores.append({'modelo': 'TF-IDF + MLP', 'accuracy': accuracy})

/home/aluno/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.8818 - loss: 0.2465
Epoch 2/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.9972 - loss: 0.0118
Epoch 3/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.9997 - loss: 0.0015
Epoch 4/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 1.0000 - loss: 3.5993e-04
Epoch 5/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 1.0000 - loss: 1.3788e-04
Epoch 6/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 1.0000 - loss: 5.6498e-05
Epoch 7/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 1.0000 - loss: 4.8476e-05
Epoch 8/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 1.0000 - loss: 3.4872e-05
Epoch 9/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 1.0000 - loss: 2.7927e-05
Epoch 10/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 1.0000 - loss: 1.7901e-05
Epoch 11/150
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 1.0000 - loss: 8.8714e-06
Epo

# 4. Comparação dos modelos

In [23]:
fig = px.bar(
    pd.DataFrame(scores),
    x='modelo',
    y='accuracy',
    width=600,
    height=400,
    template='simple_white',
    text='accuracy',
    text_auto='.1%',
)
fig.update_layout(bargap=0.2)
fig.show()